# Processus CMJ (Crump - Mode - Jagers) : exemples

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

## Processus de Poisson avec birth rate et type dépendant de l'âge

$b$ une fonction définie sur $\mathbb{R}_+$ : $\textit{birth rate}$

$\mathbb{S}$ représente l'ensemble des longueurs de télomères (à la naissance). L'unité utilisée est le millier de paires de bases (kpb). On suppose que les longueurs de télomères à la naissance sont comprises entre 5 kpb et 15 kpb. 

Semi-groupe markovien $p_t$ de $\mathbb{S}$ vers $\mathbb{S}$ :
$$ p_t(s_0, \bullet) \sim \mathcal{N}(s_0 + \alpha t -  m, \sigma) $$ (loi conditionnée à appartenir au segment $[5,15]$)

On impose $\alpha = 0.01$ (vitesse de croissance des télomères des gamètes du père : environ 20 pb/an) et on fixe $m = 35\times \alpha$ de telle sorte que la moyenne de la gaussienne soit égale à 10kpb pour $s_0 = 10$ et $t = 35$.

$\xi$ : mesure aléatoire de Poisson sur $\mathbb{S}\times \mathbb{R}_{+}$ d'intensité $b(t)p_t(s_0, \mathrm{d}s)\mathrm{d}t$

In [ ]:
class Individual:
    def __init__(self, s, t, index):
        self.s = s
        self.birth_date = t
        self.label = index
    def __repr__(self):
        s = "Individu de type {} né au temps {}".format(self.s, self.birth_date)
        return s
    def get_age(self, t):
        return max(0, t - self.birth_date)

In [ ]:
### Paramètres du modèle ###

df = pd.read_csv("fm_fecondite_age_mere.csv", index_col = 0, 
                 names = range(14,51),
                 engine = 'python', skiprows = 5, skipfooter = 4)
del df[14]

## Paramètres de simulation ##

birth_rates = np.array(df[df.index == 2000] / 10000)[0]
def birth_rate(t):
    n = int(t)-15
    if n < 0 or n > 35:
        return 0
    else:
        if n+1 == 36:
            return birth_rates[n]*(n+1-t-15)
        return birth_rates[n] + (birth_rates[n+1] - birth_rates[n])*(t-n-15)
ind = 14
s1 = np.sum(birth_rates[:ind])
s2 = np.sum(birth_rates[ind+1:])
rho = 0.5*((s2-s1)/birth_rates[ind]+1)

## Paramètres pour le noyau de transition ##

alpha = 0.015
m = alpha * (15 + ind + rho)
sigma = 0.1

def plot_birth_rate(year, color = 'b'):
    br = np.array(df[df.index == year] / 10000)[0]
    plt.plot(np.array(df.columns), br, color = color, label = "Année {}".format(year))

def p(t, s0):
    '''
    Semi-groupe de transition: simulation du type du descendant
    t : âge du parent à la reproduction
    s0 : type de départ
    '''
    x = s0 + alpha * t - m + sigma * np.random.randn()
    k = 0
    while x > 15 or x < 5:
        x = s0 + alpha * t - m + sigma * np.random.randn()
        k += 1
#         if k > 1000:
#             print("Impossible de simuler le type")
#             print(t, s0)
#             break
    return x

def random_poisson_measure_inhomogeneous(t0, T, birth_rate, sup_birth_rate, s0 = 10):
    '''
    Simulation des points pour le processus de Poisson:
    - t0 : date de naissance du parent
    - T : date de fin (pour la simulation)
    - birth_rate : fonction birth rate
    - sup_birth_rate : norme sup de la fonction birth rate (utile pour l'algo de rejet)
    - s0 : type du parent (défaut : 10)
    '''
    jump_times = list(T * np.random.rand(np.random.poisson(sup_birth_rate * T)) + t0)
    i = 0
    while i < len(jump_times):
        if np.random.rand() >= birth_rate(jump_times[i]-t0)/sup_birth_rate:
            del jump_times[i]
        else:
            i += 1
    jump_times.sort()
#     print(jump_times)
    types = [p(t-t0, s0) for t in jump_times]
    return jump_times, types

def simul_next_gen(population, birth_rate, sup_birth_rate, n_max = 100):
    '''
    Simulation de la génération suivante et ajout à la list population
    - population : liste de toutes les générations déjà simulées
    - birth_rate : fontion taux de naissance
    - sup_birth_rate : sup de la fonction taux de naissance
    - n_max : taille maximale de chaque génération
    '''
    new_gen = []
    for ind in population[-1]:
        jump_times, types = random_poisson_measure_inhomogeneous(ind.birth_date, 55, birth_rate, sup_birth_rate, ind.s)
        for i in range(len(jump_times)):
            new_gen.append(Individual(types[i], jump_times[i], ind.label + chr(i+1)))
    if len(new_gen) > n_max:
        new_gen = np.array(new_gen)[np.random.choice(range(len(new_gen)), n_max, replace = False)]
    population.append(new_gen)
    return population

def simul_pop(population, N, birth_rate, sup_birth_rate, n_max = 100):
    '''
    Simulation des N générations suivante et ajout à la list population
    - population : liste de toutes les générations déjà simulées
    - N : nombre de génération à simuler
    - birth_rate : fontion taux de naissance
    - sup_birth_rate : sup de la fonction taux de naissance
    - n_max : taille maximale de chaque génération
    '''
    for k in tqdm(range(N)):
        population = simul_next_gen(pop, birth_rate, sup_birth_rate, n_max)

In [ ]:
# Plot populations

# Simple scatter plot
def scatter_plot_population(population):
    '''
    Plot simple d'une population après simulation
    - population : liste des générations avec les individus (type et date de naissance)
    '''
    plt.figure(figsize=(10,10))
    for k in range(len(population)):
        plt.scatter([x.birth_date for x in population[k]], [x.s for x in population[k]], s=10)
    plt.xlabel("Time (birth)")
    plt.ylabel("LTL (kpb)")
    plt.title("Plot simple de la population")
    plt.show()
    
# Selection
def select(population, selection, t_min, t_max):
    '''
    Sélection des individus en fonction de leur date de naissance
    - population : liste des générations
    '''
#     selection.clear()
    for k in range(len(population)):
        print(k)
        for ind in population[k]:
            print(ind)
            d = ind.birth_date
            if d < t_max and d >= t_min:
                selection.append(ind)

# Max time
def max_time(population):
    '''
    Temps de naissance maximum = dernier temps de naissance
    - population : liste des générations
    '''
    m = 0
    for k in range(len(population)):
        for ind in population[k]:
            m = max(m, ind.birth_date)
    return m

In [ ]:
plt.figure(figsize=(7,5))
plot_birth_rate(1950, 'y')
plot_birth_rate(1975, 'g')
plot_birth_rate(2000, 'r')
plot_birth_rate(2018, 'b')
plt.xlabel('Age')
plt.ylabel('Fertility rate per year')
plt.legend()
plt.show()

In [ ]:
# Initialisation
t0 = 0
s0 = 10
n_max = 1000
pop = [[Individual(s0, t0, chr(0)) for k in range(10)]]
# sel = pop[0]

In [ ]:
# plt.figure(figsize=(5,5))
# plt.plot([len(pop[k]) for k in range(len(pop))])
# plt.xlabel("Temps (années)")
# plt.ylabel("Taille de la population")
# plt.show()

In [ ]:
# scatter_plot_population(pop)

In [ ]:
from bqplot import LinearScale, Bins, Axis, Figure, Scatter
import ipywidgets as wdgts

In [ ]:
slider = wdgts.IntSlider(0,0, len(pop)-1)

In [ ]:
n_it = wdgts.HBox([wdgts.Label("Nombre d'itérations supplémentaires : "), wdgts.IntText(value=0, description="", disabled=False)])
button1 = wdgts.ToggleButton(value=False, description='Go!', disabled=False, button_style='')
button2 = wdgts.ToggleButton(value=False, description='Reset', disabled=False, button_style='')

def update(change):
    if button1.value:
        N = n_it.children[1].value
        simul_pop(pop, N, birth_rate, 0.15, n_max = n_max)
        slider.max = len(pop)-1
        m = max_time(pop)
#         selector.max = m
    button1.value = False
    
button1.observe(update, 'value')

def clear(change):
    if button2.value:
        slider.max = 0
        pop.clear()
        pop.append([Individual(s0,t0,chr(0)) for k in range(10)])
    button2.value = False

button2.observe(clear, 'value')

In [ ]:
# m = max_time(pop)
# selector = wdgts.IntRangeSlider(value=[0, m+100], min=0, max=m+100, step=1, description='', disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')

# def selection_update(change):
#     r = change['new']
#     select(pop, sel, r[0], r[1])
#     print(sel)

# selector.observe(selection_update, 'value')

In [ ]:
x_data = [x.birth_date for x in pop[0]]
y_data = [x.s for x in pop[0]]

In [ ]:
x_sc = LinearScale()
y_sc = LinearScale()
hist = Bins(sample=y_data, scales={'x': x_sc, 'y': y_sc}, density = False, bins = int((2*len(y_data))**(1/3)))
ax_x = Axis(scale=x_sc, tick_format='0.2f', label = 'LTL (kpb)')
ax_y = Axis(scale=y_sc, orientation='vertical')

fig1 = Figure(marks=[hist], axes=[ax_x, ax_y], padding_y=0, layout=wdgts.Layout(width='600px', height='500px'), title = 'Répartition des longueurs de télomères')

In [ ]:
def foo(change):
    hist.sample = [x.s for x in pop[change['new']]]
    hist.bins = int((2*len(pop[change['new']]))**(1/3))
slider.observe(foo, 'value')

In [ ]:
wdgts.HBox([fig1,wdgts.VBox([n_it, button1, slider, button2])])